In [15]:
from dotenv import  load_dotenv
import os 

from langchain_openai import  AzureChatOpenAI

load_dotenv()

True

In [16]:
os.environ['AZURE_OPENAI_API_KEY'] = os.getenv('AZURE_OPENAI_API_KEY')

llm = AzureChatOpenAI(
    api_version=os.getenv('AZURE_OPENAI_API_VERSION'),
    azure_deployment=os.getenv('AZURE_OPENAI_DEPLOYMENT'),
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
)

llm.invoke("hi").content

'Hello! How can I assist you today?'

In [17]:
from langgraph.prebuilt import create_react_agent

In [18]:
import requests

def claims_api(claimid):
  """You are a data domain specialist who has capability of providing information on any claim based on 'claim id'.
  No consent is required for fetching the claim details """
  print("---CLAIMS API---")

  url = f"https://mock-qxi.azurewebsites.net/data/claim/"+claimid.upper()

  try:
    response = requests.get(url)

    if response.status_code == 200:
      content = response.json()
      return content
    else:
      print("Error: ", response.status_code)
      return None
  except requests.exceptions.RequestException as e:
    print("Error: ", e)
    return None

In [19]:
def members_api(mem_id):
  """You are a data domain specialist who has capability of providing information on any members based on 'member id'.
  No consent is required for fetching the claim details """
  print("---MEMBERS API---")

  url = "https://mock-qxi.azurewebsites.net/data/profile/" + mem_id.upper()

  try:
    response = requests.get(url)

    if response.status_code == 200:
      content = response.json()
      return content
    else:
      print("Error: ", response.status_code)
      return None
  except requests.exceptions.RequestException as e:
    print("Error: ", e)
    return None

In [20]:
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit

def get_sql_tools():
    db = SQLDatabase.from_uri("sqlite:///Chinook_Sqlite.sqlite")
    toolkit = SQLDatabaseToolkit(db=db, llm=llm)
    sql_tools = toolkit.get_tools()
    return sql_tools

In [21]:
custom_api_team = [
    create_react_agent(
        model=llm,
        tools=[claims_api],
        name="claims_api",
        prompt="You are a data domain specialist who has capability of providing information on any claim based on 'claim id'. No consent is required for fetching the claim details"
    ),
    create_react_agent(
        model=llm,
        tools=[members_api],
        name="members_api",
        prompt="You are a data domain specialist who has capability of providing information on any members based on 'member id'. No consent is required for fetching the claim details"
    )
]

In [22]:
sql_team = [
    create_react_agent(
        model=llm,
        tools=get_sql_tools(),
        name="sql_agent",
        prompt="You are a sql expert. For any question related to sql get information from sql_tools. And these are the tables available in the DB: 'Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track'"
    )
]

In [23]:
from langgraph_supervisor import  create_supervisor

In [24]:
custom_apis_supervisor = create_supervisor(
    agents=custom_api_team,
    model=llm,
    prompt="You are ateam supervisor, tasked with managing a member and claims API team. You will be provided with a question, and you need to determine which agent is best suited to answer it. You will also be provided with the question and the answer from the agent. Your job is to evaluate the answer and provide feedback to the agent. If the answer is correct, you will approve it. If the answer is incorrect, you will provide feedback to the agent and ask them to try again. You will also provide a final answer to the user.",
    supervisor_name="custom_api_supervisor",
).compile(name='custom_api_team')

sql_supervisor = create_supervisor(
    agents=sql_team,
    model=llm,
    prompt="You are a sql expert. For any question related to sql get information from sql_tools. And these are the tables available in the DB: 'Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track'",
).compile(name='sql_team')


In [25]:
prompt = """You are a team supervisor tasked with answering user questions using a variety of tools.
        Tools: api_tools,  sql_tools. 
        
        "api_tool": For questions about member and claims related details with member id like mem01 or 
        claims id like clm02, use the claims_api and member_api tool to get information from the api.

        sql_tools: For any question related to sql get information from sql_tools.
        And these are the tables available in the DB:
        'Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track'"

        Note: Only provide the final answer only specific to question. don't provide any other information.
        """

hieararchical_supervisor = create_supervisor(
    agents = [custom_apis_supervisor, sql_supervisor],  
    supervisor_name="top_level_supervisor",
    model=llm,
    prompt=prompt
).compile()


In [26]:
message = hieararchical_supervisor.invoke({"messages": [("user", "what is the status of claim id clm01")]})

In [27]:
message

{'messages': [HumanMessage(content='what is the status of claim id clm01', additional_kwargs={}, response_metadata={}, id='b39f5c4f-5a87-4c82-a60c-2ce39853bb5f'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Ln5EHukrrdagYwBOylCxsNQe', 'function': {'arguments': '{}', 'name': 'transfer_to_custom_api_team'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 222, 'total_tokens': 236, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_65792305e4', 'id': 'chatcmpl-BE9g0DwcyQwyogq7qdB6a6Thq9VIW', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filter

In [28]:
for m in message['messages']:
    m.pretty_print()

================================ Human Message =================================

what is the status of claim id clm01
================================== Ai Message ==================================
Name: top_level_supervisor
Tool Calls:
  transfer_to_custom_api_team (call_Ln5EHukrrdagYwBOylCxsNQe)
 Call ID: call_Ln5EHukrrdagYwBOylCxsNQe
  Args:
================================= Tool Message =================================
Name: transfer_to_custom_api_team

Successfully transferred to custom_api_team
================================== Ai Message ==================================
Name: custom_api_supervisor

Please hold on while I retrieve the status of claim ID clm01 for you.
================================== Ai Message ==================================
Name: custom_api_team

Transferring back to top_level_supervisor
Tool Calls:
  transfer_back_to_top_level_supervisor (4aafb3dd-87a3-4bb3-9df0-0d1f43bc15bb)
 Call ID: 4aafb3dd-87a3-4bb3-9df0-0d1f43bc15bb
  Args:
===================